In [5]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from lib.data import prepare_input # TODO: replace with utils.preprocess_function which separates question and headers by separating token instead of prefixes.

In [6]:
tokenizer = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema") # TODO: replace with own tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema") # TODO: replace with own model

In [7]:
def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table, tokenizer=tokenizer)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, max_length=700)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

In [8]:
print(inference(question="Get all names of people that are 25 years old", table=["id", "name", "age"]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT name FROM table WHERE age = 25
